In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
!wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

--2024-06-25 00:07:57--  https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1115394 (1.1M) [text/plain]
Saving to: ‘input.txt’

input.txt           100%[===================>]   1.06M  --.-KB/s    in 0.006s  

2024-06-25 00:07:57 (185 MB/s) - ‘input.txt’ saved [1115394/1115394]



In [3]:
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2

In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
vocab_size

65

In [5]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [6]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]


In [7]:
#define the get batch function
def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))  #for each example, we'll index seq_length (block size) and stack as batches
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+1+block_size] for i in ix])
  x, y = x.to(device), y.to(device)
  return x, y

In [8]:
# loss calculation
@torch.no_grad()     #deactivates keeping track of gradient enables faster computation.
def estimate_loss():
  out = {}
  model.eval()   #deactivates dropout and batchnorm
  for split in ['train', 'val']:
    losses = torch.zeros(eval_iters)
    for k in range(eval_iters):
      X, Y = get_batch(split)
      logits, loss = model(X, Y)
      losses[k] = loss.item()         #for mean purposes
    out[split] = losses.mean()
  model.train()                       #set back in training mode to enable those layers
  return out

In [9]:
# #model ++

# # one self attention head
# class Head(nn.Module):
#   def __init__(self, head_size):       #headsize equals model dimension / number of heads
#     super().__init__()
#     self.key = nn.Linear(n_embd, head_size, bias=False)
#     self.query = nn.Linear(n_embd, head_size, bias=False)
#     self.value = nn.Linear(n_embd, head_size, bias=False)
#     self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
#     self.dropout = nn.Dropout(dropout)

#   def forward(self, x):

#     B, T, C = x.shape     #batch, time, channel
#     k = self.key(x)       #B, T, hs
#     q = self.query(x)     #B, T, hs

#     wei = q @ k.transpose(-2, -1) * C**-0.5  #Weight decay  (B, T, T)
#     wei = wei.masked_fill(self.tril[:T, :T] == 0, float('inf')) # B, T, T
#     wei = F.softmax(wei, dim=-1)
#     wei = self.dropout(wei)   #attending to only previous tokens

#     #perform weighted aggregation with values
#     v = self.value(x)   #B, T, hs
#     out = wei @ v        #B, T, hs
#     return out


# class MultiHeadAttention(nn.Module):
#   def __init__(self, num_heads, head_size):
#     super().__init__()
#     self.heads = nn.ModuleList(Head(head_size) for _ in range(num_heads))
#     self.proj = nn.Linear(head_size * num_heads, n_embd)
#     self.dropout = nn.Dropout(dropout)

#   def forward(self, x):
#     out = torch.cat([h(x) for h in self.heads], dim=-1)  #B, T, hs * no_of_heads
#     out = self.dropout(self.proj(out))   # B, T, C
#     return out


# class FeedFoward(nn.Module):
#   def __init__(self, n_embd):
#     super().__init__()
#     self.net = nn.Sequential(
#         nn.Linear(n_embd, 4 * n_embd),
#         nn.ReLU(),
#         nn.Linear(4 * n_embd, n_embd),
#         nn.Dropout(dropout)
#     )

#   def forward(self, x):
#     return self.net(x)

# class Block(nn.Module):
#   def __init__(self, n_embd, n_head):
#     super().__init__()
#     head_size = n_embd // n_head      # n_embd == Dmodel
#     self.sa = MultiHeadAttention(n_head, head_size)
#     self.ffwd = FeedFoward(n_embd)
#     self.ln1 = nn.LayerNorm(n_embd)
#     self.ln2 = nn.LayerNorm(n_embd)

#   def forward(self, x):
#     x = x + self.sa(self.ln1(x))   # making the residual connection backpropagatable
#     x = x + self.ffwd(self.ln2(x)) # as opposed to x=self.msa(self.ln(x) + x)
#     return x


# class GPTLanguageModel(nn.Module):
#   def __init__(self):
#     super().__init__()
#     self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
#     self.position_embedding_table = nn.Embedding(block_size, n_embd)
#     self.blocks = nn.Sequential(*[Block(n_head, n_embd) for _ in range(n_layer)])  # argument unparking
#     self.ln_f = nn.LayerNorm(n_embd)
#     self.lm_head = nn.Linear(n_embd, vocab_size)

#     self.apply(self._init_weights)

#   def _init_weights(self, module):
#     if isinstance(module, nn.Linear):
#       torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
#       if module.bias is not None:
#         torch.nn.init.zeros_(module.bias)
#     elif isinstance(module, nn.Embedding):
#       torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

#   def forward(self, idx, targets=None):
#     B, T = idx.shape

#     # Attention computation
#     tok_emb = self.token_embedding_table(idx)   # B, T, C
#     pos_emb = self.position_embedding_table(torch.arange(T, device=device))  #the arange tensor is not part of the model, it has to pass to device  B, T
#     x = tok_emb + pos_emb     # B, T, C
#     x = self.blocks(x)        # B, T, C
#     x = self.ln_f(x)          # B, T, C
#     logits = self.lm_head(x)  # B, T, vocab_size

#     if targets is None:
#       loss = None
#     else:
#       B, T, C = logits.shape  #crossentropy requires logits(N, C) and target(N)
#       logits = logits.view(B*T, C)
#       targets = targets.view(B*T)
#       loss = F.cross_entropy(logits, targets)
#     return logits, loss


#   def generate(self, idx, max_new_tokens):

#     for _ in range(max_new_tokens):    # B, T   we start with a single token

#       #crop to predict from block size for max context
#       idx_cond = idx[:, -block_size:]

#       logits, loss = self(idx_cond)
#       #last step
#       logits = logits[:, -1, :]  #B, C   taking the last dim
#       probs = F.softmax(logits, dim=-1)  #B, C
#       idx_next = torch.multinomial(probs, num_samples=1)  #B, 1   (C is the probability distribution)
#       idx = torch.cat((idx, idx_next), dim=1)  #B, T+1
#     return idx


# model = GPTLanguageModel()
# model.to(device)






In [ ]:
class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


model = GPTLanguageModel()
model.to(device)


In [11]:
#printing model parameters
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

#creating optimizer object
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

10.788929 M parameters


In [15]:
# training loop

for i in range(max_iters):
  # evaluation time
  if i % eval_interval == 0 or i == max_iters - 1:
    losses = estimate_loss()
    print(f"step {i}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()


step 0: train loss 3.3120, val loss 3.3548
step 500: train loss 3.3094, val loss 3.3516
step 1000: train loss 3.3088, val loss 3.3481
step 1500: train loss 3.3010, val loss 3.3445
step 2000: train loss 3.2969, val loss 3.3394
step 2500: train loss 3.2887, val loss 3.3290
step 3000: train loss 3.2774, val loss 3.3227
step 3500: train loss 3.2705, val loss 3.3111
step 4000: train loss 3.2606, val loss 3.3011
step 4500: train loss 3.2451, val loss 3.2863
step 4999: train loss 3.2290, val loss 3.2682


In [16]:
#inference

with torch.inference_mode():
  context = torch.zeros((1, 1), dtype=torch.long, device=device)
  #open('shakespare_more', 'w').write(decode(model.generate(context, max_new_tokens=1000)[0].tolist()))  #can generate a lot but context is limited to block_size
  print(decode(model.generate(context, max_new_tokens=1000)[0].tolist()))



o
 gliAe uKwope.,d ubo
tmttT,bE,u' t; cit, fuke legy'aihfotafi t e serhi a
h iea pun:toouupntiise sU b ga  ufn- de ,e.ogugo  mh hr
anaA atiy henega tsifck v h ip : usu o tbuot bsh  ifalnerudbeleIw 
o edt t sr sa el sirh tfi rheZn.hfttmofanddavrfrmmef n t,hitee rd
 un nko

oyt
opot nth t nut
na d  U
raoh ,rbenu  d, r orasec

hI ytratdidh Ohiuw t  swiS uce uh atau!rA wah R ta,Mtt.Cgh   
ox s r? kntimiSvuRaro:fw teebpg iw niseregyc,e eybfrrtR ep
aso hheneet recoes
te :tmeDBini gre dgfeWoou ms  eeGfgofThli roru  hs, t t lubnu   mndneieeohheteisqbmEatoyoisniaugr r Nfr'dntb
ii,hnan 
dotisoreslei o w pt rpenfGdxeioh ar. m:e sieoO,ye
opV: turentoaruat soahy,arei
tth o  b wmat U  dE t ,wih nt:hac
 t o ahtrEe,uh ho!, : n toln i dtptir oor dq,etoW ':Th tL onetsesbAerornhan
o':,s innesimC  aeTw ,:o
ouysS:ateEurIuofCfvirp
 h Al&we  atn  gsI zbsge atkes wY iorbUl yntofd ehee, mh e t hk d,neinr

 oetlr,it!isiriun Anhhola w !A th hitdeDas. dyul lr ,b ie eLmrooPga  eaisetonr:tel oftht weyseiataeast  d